<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer: Is the layer that takes in the data
### Hidden Layer: A added layer(s) in between the input and output layer
### Output Layer: The layer that gives us our output
### Neuron: holds the data
### Weight: All data is multiplied by a weight
### Activation Function: The function that determines whether or not a neuron fires
### Node Map:
### Perceptron: A simple input, hidden, output neural network. 


## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### Your Answer Here

## Write your own perceptron code that can correctly classify a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [65]:
import numpy as np
import pandas as pd

In [292]:
class Perceptron(object):
    def __init__(self, input, output, bias):
        self.input = input
        self.output = output
        self.weights = np.random.random((len(self.input.T),1)) - 1
        self.bias = bias
    
    def sigmoid(self, x):
        return 1/(1+np.exp(-x))
    
    def sigmoid_derivative(self, x):
        sx = self.sigmoid(x)
        return sx * (1-sx)
    
    def train(self, epochs):
        for num in range(epochs):
            weighted_sum = np.dot(self.input, self.weights)
            activated_output = self.sigmoid(weighted_sum)
            error = self.output - activated_output
            adjustments = error * self.sigmoid_derivative(activated_output)
            weights += np.dot(self.input.T, adjustments)
    
    def predict(self, X):
        return np.where(self.sigmoid(np.dot(X, self.weights)) >= .5, 0, 1)      

In [293]:
X = np.array([[0,0], [1,0], [0,1], [1,1]])
y = [[1], [1], [1], [0]]

In [294]:
neural_net = NeuralNetwork(X, y, 1)

In [298]:
neural_net.weights

array([[-0.69007961],
       [-0.69007961]])

In [296]:
neural_net.train(10000)

In [297]:
neural_net.weights

array([[-0.69007961],
       [-0.69007961]])

In [236]:
neural_net.predict([0,1])

array([1])

In [302]:
weights = np.random.random((2, 1))
print(f"Weights before training: {weights}")
def new_weights
for i in range(10000):
    
    weighted_sum = np.dot(X, weights)
    
    activated_output = sigmoid(weighted_sum)
    
    error = y - activated_output
    
    adjustments = error * sigmoid_derivative(activated_output)
    
    weights += np.dot(X.T, adjustments)

print(f"Weights After training: {weights}")

Weights before training: [[0.07180693]
 [0.66686571]]
Weights After training: [[-3.05311332e-16]
 [ 1.24900090e-16]]


## Implement your own Perceptron Class and use it to classify a binary dataset like: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 
- [Titanic](https://raw.githubusercontent.com/ryanleeallred/datasets/master/titanic.csv)
- [A two-class version of the Iris dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/Iris.csv)

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [186]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [187]:
iris = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/Iris.csv")

In [188]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [189]:
features = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]
target = "Species"

In [190]:
scaler = StandardScaler()
encoder = OrdinalEncoder()

#Ordinally encoding target
encoder.fit(np.array(iris[target]).reshape(-1, 1))
y_encoded = encoder.fit_transform(np.array(iris[target]).reshape(-1, 1))

#standardizing features
scaler.fit(iris[features])
X_scaled = scaler.fit_transform(iris[features])

In [191]:
iris["SepalLengthCm"], iris["SepalWidthCm"], iris["PetalLengthCm"], iris["PetalWidthCm"] = X_scaled.T
iris[target] = y_encoded

In [192]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,-0.900681,1.032057,-1.341272,-1.312977,0.0
1,2,-1.143017,-0.124958,-1.341272,-1.312977,0.0
2,3,-1.385353,0.337848,-1.398138,-1.312977,0.0
3,4,-1.506521,0.106445,-1.284407,-1.312977,0.0
4,5,-1.021849,1.263460,-1.341272,-1.312977,0.0


In [260]:
X = np.array(iris[features])
y = np.array(iris[[target]])

In [261]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size= .8, test_size = .2)

In [262]:
def sigmoid(x):
        return 1/(1+np.exp(-x))
    
def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1-sx)

In [263]:
X_train.shape, y_train.shape

((120, 4), (120, 1))

In [285]:
weights = np.random.random((4, 1))
print(f"Weights before training: {weights}")
for i in range(10000):
    
    weighted_sum = np.dot(X_train, weights)
    
    activated_output = sigmoid(weighted_sum)
    
    error = y_train - activated_output
    
    adjustments = error * sigmoid_derivative(activated_output)
    
    weights = np.dot(X_train.T, adjustments)

print(f"Weights After training: {weights}")

Weights before training: [[0.16073123]
 [0.02334471]
 [0.51667722]
 [0.16283465]]
Weights After training: [[ 5.83378676]
 [-3.3786681 ]
 [ 7.21215658]
 [ 7.59572436]]
